In [1]:
!pip install mlflow dagshub -q
!pip install pyngrok -q
!pip install optuna -q
!pip install efficientnet -q
!pip install ensure

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import re
import requests
import csv
import tensorflow as tf
import math
import random


from sklearn.metrics import confusion_matrix,  multilabel_confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from keras.applications import VGG16
import efficientnet.keras as efn

from keras.utils import img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Dropout
from keras.callbacks import ModelCheckpoint , EarlyStopping
from keras.utils import Sequence
from keras.layers import Dense, Flatten, Dropout, BatchNormalization,GlobalAveragePooling2D
from keras.models import Model
from keras import layers
from keras import models

2025-06-29 13:19:23.915188: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751203164.340211      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751203164.461246      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Utility

In [3]:
import yaml
from box import ConfigBox
from ensure import ensure_annotations
from box.exceptions import BoxValueError
from pathlib import Path
import yaml
import os

@ensure_annotations
def read_yaml_file(file_path:Path):

    """reads yaml file and returns

    Args:
        path_to_yaml (str): path like input

    Raises:
        ValueError: if yaml file is empty
        e: empty file

    Returns:
        ConfigBox: ConfigBox type
    """
    with open(file_path, 'r') as file:
        try:
            yaml_data = yaml.safe_load(file)
            return ConfigBox(yaml_data)
        except yaml.YAMLError as e:
            print(f"Error reading YAML file {file_path}: {e}")

import yaml

@ensure_annotations
def write_yaml_file(data:dict, file_path:Path):
    """

    """
    with open(file_path, 'w') as file:
        try:
            yaml.dump(data, file, default_flow_style=False)
            print(f"Data successfully written to {file_path}")
        except yaml.YAMLError as e:
            print(f"Error writing YAML file {file_path}: {e}")



In [4]:
import pickle
@ensure_annotations
def save_pkl(data:dict, file_path:Path):
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)
        print(f"Pickle file saved to {file_path}")

# Example usage:
my_data = {'a': 1, 'b': 2}
save_pkl(my_data, Path('data.pkl'))


@ensure_annotations
def load_pkl(file_path:Path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
        return data

# Example usage:
loaded_data = load_pkl(Path('data.pkl'))
print(loaded_data)


Pickle file saved to data.pkl
{'a': 1, 'b': 2}


In [5]:
import joblib

def save_joblib(data, file_path):
    joblib.dump(data, file_path)
    print(f"Joblib file saved to {file_path}")

# Example usage:
my_model = {'model': 'example'}
save_joblib(my_model, 'model.joblib')


def load_joblib(file_path):
    data = joblib.load(file_path)
    return data

# Example usage:
loaded_model = load_joblib('model.joblib')
print(loaded_model)


Joblib file saved to model.joblib
{'model': 'example'}


# YAML

In [6]:
yaml_data={
"rotation_range":30,
"width_shift_range":0.2,
"height_shift_range":0.2,
"shear_range":0.2,
"zoom_range":0.2,
"horizontal_flip":True

}

write_yaml_file(yaml_data, Path('preprocess_config.yaml'))

Data successfully written to preprocess_config.yaml


In [7]:
yaml_data={
"experiment_name":"document_image_MLclassification" ,
"run_name" :"cnn_model",
"epochs": 100,
"batch_size":64,
"learning_rate": 0.001,
"optimizer": "adam",
"loss_function": "binary_crossentropy",
"dropout_rate": 0.2,
"hidden_units": [1024],
"input_shape": [224,224, 3]
}

write_yaml_file(yaml_data, Path('cnn_model_config.yaml'))

Data successfully written to cnn_model_config.yaml


In [8]:
yaml_data={
"experiment_name":"document_image_classification" ,
"run_name" :"vgg_model",
"epochs": 100,
"batch_size":64,
"learning_rate": 0.001,
"optimizer": "adam",
"loss_function": "categorical_crossentropy",
"dropout_rate": 0.2,
"hidden_units": [1024],
"input_shape": [224,224, 3]
}

write_yaml_file(yaml_data, Path('vgg16_model_config.yaml'))

Data successfully written to vgg16_model_config.yaml


In [9]:
yaml_data={
"experiment_name":"document_image_classification" ,
"run_name" :"eff_model",
"epochs": 100,
"batch_size":64,
"learning_rate": 0.001,
"optimizer": "adam",
"loss_function": "categorical_crossentropy",
"dropout_rate": 0.2,
"hidden_units": [512],
"input_shape": [224,224, 3]
}

write_yaml_file(yaml_data, Path('eff_model_config.yaml'))

Data successfully written to eff_model_config.yaml


In [10]:
import os
from datetime import datetime
from pathlib import Path

def save_model_to_artifacts(experiment_id: str) -> str:
    # Create timestamp string: YYYY-MM-DD_HH-MM-SS
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    # Construct full path
    save_path = Path("artifacts")/ experiment_id / timestamp
    save_path.mkdir(parents=True, exist_ok=True)  # Create all directories

    # Define full model path
    model_path = save_path

    return str(model_path)


# Entity

In [11]:
#%%writefile entity.py
from typing import Tuple,List
#from constant import *
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
  """
  Data intection config
  """
  train_path:Path
  test_path:Path
  train_split_size:float


@dataclass(frozen=True)
class PreprocessConfig:
  """
  Data preprocess config
  """
  rotation_range:int
  width_shift_range:float
  height_shift_range:float
  shear_range:float
  zoom_range:float
  horizontal_flip:bool

@dataclass(frozen=True)
class ModelBuildConfig:
  """
  Model build config
  """
  input_shape:Tuple[int,int,int]
  len_class:int



@dataclass(frozen=True)
class ModelTrainerConfig:
  """
  Model build config
  """
  experiment_name:str
  run_name:str
  epochs:int
  batch_size:int
  learning_rate:float
  optimizer:str
  loss_function:str
  dropout_rate:float
  hidden_units:List[int]
  input_shape:list


@dataclass(frozen=True)
class MlflowConfig:
  """
  mlflow config
  """
  mlflow_tracking:str
  repo_owner:str
  repo_name:str


# configuration

In [12]:
#%%writefile constant.py

from google.colab import userdata
from pathlib import Path

# Data ingestion
TRAIN_DS_PATH:Path=Path('/kaggle/input/text-document-images/train')
TEST_DS_PATH:Path=Path('/kaggle/input/text-document-images/test')
TRAIN_SPLIT_SIZE=0.8
## preprocess
PREPROCESS_CONFIG_PATH:Path=Path('/kaggle/working/preprocess_config.yaml')

## Model training
CNN_MODEL_CONFIG_PATH:Path=Path('/kaggle/working/cnn_model_config.yaml')
VGG_MODEL_CONFIG_PATH:Path=Path('/kaggle/working/vgg16_model_config.yaml')
EFF_MODEL_CONFIG_PATH:Path=Path('/kaggle/working/eff_model_config.yaml')

# Model Build
INPUT_SHAPE=(224,224,3)
CLASS_LENGTH=5


##MLFLOW
MLFLOW_TRACKING="https://dagshub.com/kaushigihanml/document_classification.mlflow"
REPO_OWNER='kaushigihanml'
REPO_NAME='document_classification'


In [13]:
#%%writefile cofiguration.py

from google.colab import userdata
from pathlib import Path

class ConfigManager():

  def __init__(self,train_path:Path=TRAIN_DS_PATH,test_path:Path=TEST_DS_PATH,train_split_size:float=TRAIN_SPLIT_SIZE,
               preprocess_congig_path:Path=PREPROCESS_CONFIG_PATH,input_shape:Tuple[int,int,int]=INPUT_SHAPE,
               mlflow_tracking:str=MLFLOW_TRACKING,repo_owner:str=REPO_OWNER,repo_name:str=REPO_NAME,
               class_length:int=CLASS_LENGTH,cnn_model_trainer_config_path:Path=CNN_MODEL_CONFIG_PATH,vgg_model_trainer_config_path:Path=VGG_MODEL_CONFIG_PATH,
               eff_model_trainer_config_path:Path=EFF_MODEL_CONFIG_PATH
               ):
    #data ingestion
    self.train_path=train_path
    self.test_path=test_path
    self.train_split_size=train_split_size

    #Model Build
    self.input_shape=input_shape
    self.class_length=class_length

    #MLFlow setup
    self.mlflow_tracking=mlflow_tracking
    self.repo_owner=repo_owner
    self.repo_name=repo_name

    #YAML file path
    self.preprocess_congig_path=preprocess_congig_path
    self.cnn_model_trainer_config_path=cnn_model_trainer_config_path
    self.vgg_model_trainer_config_path=vgg_model_trainer_config_path
    self.eff_model_trainer_config_path=eff_model_trainer_config_path

    # model trainer config
    self.proprocess_config=read_yaml_file(self.preprocess_congig_path)
    self.cnn_model_trainer_config=read_yaml_file(self.cnn_model_trainer_config_path)
    self.vgg_model_trainer_config=read_yaml_file(self.vgg_model_trainer_config_path)
    self.eff_model_trainer_config=read_yaml_file(self.eff_model_trainer_config_path)


  def get_data_ingestion_config(self) -> DataIngestionConfig:
    try:

        data_ingestion_config = DataIngestionConfig(
            train_path=self.train_path,
            test_path=self.test_path,
            train_split_size=self.train_split_size

        )

        return data_ingestion_config
    except Exception as e:
      raise e


  def get_mlflow_config(self) -> MlflowConfig:
    try:
      mlflow_config=MlflowConfig(

      mlflow_tracking=self.mlflow_tracking,
      repo_owner=self.repo_owner,
      repo_name=self.repo_name
      )
      return mlflow_config

    except Exception as e:
      raise e



  def get_preprocess_config(self) -> PreprocessConfig:
    try:
      config=self.proprocess_config
      preprocess_config=PreprocessConfig(
          rotation_range=config.rotation_range,
          width_shift_range=config.width_shift_range,
          height_shift_range=config.height_shift_range,
          shear_range=config.shear_range,
          zoom_range=config.zoom_range,
          horizontal_flip=config.horizontal_flip
      )

      return preprocess_config

    except Exception as e:
      raise e

  def get_model_build_config(self) -> ModelBuildConfig:
    try:
      model_build_config=ModelBuildConfig(
          input_shape=self.input_shape,
          len_class=self.class_length
      )
      return model_build_config

    except Exception as e:
      raise e

  def get_cnn_model_trainer_config(self) -> ModelTrainerConfig:
    try:
      config = self.cnn_model_trainer_config
      cnn_model_trainer_config = ModelTrainerConfig(
          experiment_name=config.experiment_name,
          run_name=config.run_name,
          epochs=config.epochs,
          batch_size=config.batch_size,
          learning_rate=config.learning_rate,
          optimizer=config.optimizer,
          loss_function=config.loss_function,
          dropout_rate=config.dropout_rate,
          hidden_units=config.hidden_units,
          input_shape=config.input_shape
      )

      return cnn_model_trainer_config

    except Exception as e:
      raise e



  def get_vgg_model_trainer_config(self) -> ModelTrainerConfig:
    try:
      config = self.vgg_model_trainer_config
      vgg_model_trainer_config = ModelTrainerConfig(
          experiment_name=config.experiment_name,
          run_name=config.run_name,
          epochs=config.epochs,
          batch_size=config.batch_size,
          learning_rate=config.learning_rate,
          optimizer=config.optimizer,
          loss_function=config.loss_function,
          dropout_rate=config.dropout_rate,
          hidden_units=config.hidden_units,
          input_shape=config.input_shape
      )

      return vgg_model_trainer_config

    except Exception as e:
      raise e


  def get_eff_model_trainer_config(self) -> ModelTrainerConfig:
    try:
      config = self.eff_model_trainer_config
      eff_model_trainer_config = ModelTrainerConfig(
          experiment_name=config.experiment_name,
          run_name=config.run_name,
          epochs=config.epochs,
          batch_size=config.batch_size,
          learning_rate=config.learning_rate,
          optimizer=config.optimizer,
          loss_function=config.loss_function,
          dropout_rate=config.dropout_rate,
          hidden_units=config.hidden_units,
          input_shape=config.input_shape
      )

      return eff_model_trainer_config

    except Exception as e:
      raise e

# Connections

In [14]:
import dagshub

class MlflowConnection():
  def __init__(self,mlflow_config:MlflowConfig):
    self.mlflow_config=mlflow_config

  def mlflow_connection(self):
    dagshub.init(repo_owner=self.mlflow_config.repo_owner, repo_name=self.mlflow_config.repo_name, mlflow=True)
    mlflow.set_tracking_uri(self.mlflow_config.mlflow_tracking)

# SRC

### Ingestion

In [15]:
import pandas as pd
from typing import Tuple, List


class DataIngection():
  """
  Data intection class

  """
  def __init__(self,data_ingection_config:DataIngestionConfig):
    """
    Args:data_ingection_config:DataIngectionConfig
    """
    try:
      self.data_ingection_config=data_ingection_config
    except Exception as e:
      raise e

  @staticmethod
  def get_image_paths(path_to_subset:Path) -> List[str]:
  # Collect all valid image paths
    paths = []
    for folder in os.listdir(path_to_subset):
        folder_path = os.path.join(path_to_subset, folder)
        for image in os.listdir(folder_path):
            path_to_image = os.path.join(folder_path, image)

            # Check if image is valid
            img = cv2.imread(path_to_image)
            if img is not None:
                paths.append(path_to_image)
    return paths


  def get_file_path(self) -> Tuple[List[str], List[str], List[str]]:
      """
      Load file paths for train, validation, and test datasets.
      """
      try:
          # Get and split train, val
          paths = DataIngection.get_image_paths(path_to_subset=self.data_ingection_config.train_path)
          train_paths, val_paths = train_test_split(paths, train_size=self.data_ingection_config.train_split_size, shuffle=True, random_state=42)

          # get test paths
          test_paths = DataIngection.get_image_paths(path_to_subset=self.data_ingection_config.test_path)

          return train_paths, val_paths, test_paths

      except Exception as e:
        raise e





## Preprocess

In [16]:
from typing import Tuple, List
import numpy as np
import math
import cv2
import os
from keras.utils import Sequence
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def create_datagen_from_config(config: PreprocessConfig) -> ImageDataGenerator:
    return ImageDataGenerator(
        rotation_range=config.rotation_range,
        width_shift_range=config.width_shift_range,
        height_shift_range=config.height_shift_range,
        shear_range=config.shear_range,
        zoom_range=config.zoom_range,
        horizontal_flip=config.horizontal_flip
    )


class Dataset(Sequence):
    def __init__(self, paths: List[str], batch_size: int, input_shape: Tuple[int, int, int],
                 datagen: ImageDataGenerator = None, augment: bool = False):
        super(Dataset, self).__init__()
        self.batch_size = batch_size
        self.__data = paths
        self.image_size = input_shape
        self.augment = augment
        self.datagen = datagen

        self.classes = []
        self.label2id = {}
        self.id2label = {}
        self.mlb = None  # MultiLabelBinarizer instance

        self.__labels = self.__list_labels(self.__data)

    def __len__(self):
        return math.ceil(len(self.__data) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.__data[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.__labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        images = np.array([self.__load_image(i) for i in batch_x])
        return images, np.array(batch_y)

    def __list_labels(self, paths: List[str]):
        labels = []
        for path in paths:
            label = path.split(os.path.sep)[-2].split(' ')
            labels.append(label)

        self.mlb = MultiLabelBinarizer()
        labels_bin = self.mlb.fit_transform(labels)
        self.classes = list(self.mlb.classes_)

        # Generate label2id and id2label mappings
        self.label2id = {label: idx for idx, label in enumerate(self.classes)}
        self.id2label = {idx: label for idx, label in enumerate(self.classes)}

        return labels_bin

    """def __load_image(self, path: str):
        img = cv.imread(path)
        img = cv.resize(img, (self.image_size[0], self.image_size[1]))
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

        if self.augment:
            img = train_datagen.random_transform(img)

        img = img.astype("float32") / 255.0
        return img"""

    def __load_image(self, path: str):
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (self.image_size[1], self.image_size[0]))  # width, height

        if self.augment:
            img = self.datagen.random_transform(img)

        img = img.astype("float32") / 255.0
        #img = np.reshape(img, self.image_size)  # (height, width, channels)
        return img

## Model build

In [17]:
from tensorflow.keras import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models

class ModelBuilder():
  def __init__(self,model_build_config:ModelBuildConfig):
    self.model_build_config=model_build_config
    self.input_shape=model_build_config.input_shape
    self.len_class=model_build_config.len_class


  def cnn_model(self) -> Model:
      model = tf.keras.models.Sequential()

      model.add(Conv2D(32, 3, padding = 'same', input_shape = self.input_shape, kernel_initializer = 'he_normal', activation = 'relu'))
      model.add(BatchNormalization()) #Stack of images become stack with no negative values
      model.add(MaxPooling2D(2))
      model.add(Dropout(0.25))

      model.add(Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
      model.add(BatchNormalization())
      model.add(Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
      model.add(BatchNormalization())
      model.add(MaxPooling2D())
      model.add(Dropout(0.25))

      model.add(Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
      model.add(BatchNormalization())
      model.add(Conv2D(128, 2, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
      model.add(BatchNormalization())
      model.add(MaxPooling2D())
      model.add(Dropout(0.25))

      model.add(Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
      model.add(BatchNormalization())
      model.add(Conv2D(256, 2, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
      model.add(BatchNormalization())
      model.add(MaxPooling2D())
      model.add(Dropout(0.25))

      #model.add(Conv2D(1024, 3, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
      #model.add(BatchNormalization())
      #model.add(Conv2D(1024, 2, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
      #model.add(BatchNormalization())
      #model.add(MaxPooling2D())
      #model.add(Dropout(0.25))

      model.add(Flatten())
      model.add(Dense(1024, activation = 'relu', kernel_initializer = 'he_normal', ))
      model.add(BatchNormalization())
      model.add(Dropout(0.5))

      model.summary()
      model.add(Dense(self.len_class, activation = 'softmax'))
      return model

  def VGG16_model(self)-> Model:
      base_model = VGG16(weights='imagenet', include_top=False, input_shape=self.input_shape)
      # Add custom layers for classification with dropout
      x = base_model.output
      x = GlobalAveragePooling2D()(x)
      x = Dense(1024, activation="relu",kernel_initializer = 'he_normal')(x)
      x = BatchNormalization()(x)
      x = Dropout(0.5)(x)

      output = Dense(self.len_class, activation="sigmoid")(x)
      # Create the complete model
      model = Model(inputs=base_model.input, outputs=output)
      # Summary of the model
      model.summary()

      return model

  def EfficientNetB0_model(self)-> Model:
      base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=self.input_shape)

      # Freeze base model
      for layer in base_model.layers:
          layer.trainable = False

      model = models.Sequential()
      model.add(base_model)
      model.add(layers.GlobalAveragePooling2D())
      model.add(layers.Dense(512, activation='relu',kernel_initializer = 'he_normal', name='dense'))
      model.add(layers.Dropout(0.5))
      model.add(layers.Dense(self.len_class, activation='softmax', name='predictions'))

      model.summary()
      return model

## Model Trainer



In [18]:

from tensorflow.keras.optimizers import Adam
from mlflow.models import infer_signature
import keras
import os
import mlflow
import mlflow.tensorflow

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report




class ModelTrainer():
    def __init__(self,train_paths:List, val_paths:List, test_paths:List):
      self.train_paths=train_paths
      self.val_paths=val_paths
      self.test_paths=test_paths


    @staticmethod
    def log_training_plots(history, run_id):
        """Log training history plots to MLflow."""

        # Plot training history
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

        ax1.plot(history.history["loss"], label="Training Loss")
        ax1.plot(history.history["val_loss"], label="Validation Loss")
        ax1.set_title("Model Loss")
        ax1.set_xlabel("Epoch")
        ax1.set_ylabel("Loss")
        ax1.legend()

        ax2.plot(history.history["accuracy"], label="Training Accuracy")
        ax2.plot(history.history["val_accuracy"], label="Validation Accuracy")
        ax2.set_title("Model Accuracy")
        ax2.set_xlabel("Epoch")
        ax2.set_ylabel("Accuracy")
        ax2.legend()

        plt.tight_layout()
        plt.savefig("training_history.png", dpi=300, bbox_inches="tight")
        mlflow.log_artifact("training_history.png")
        plt.close()

    @staticmethod
    def log_evaluation_metrics_classification(model,dataset):
        """Log comprehensive evaluation metrics."""

        all_true_labels = []
        all_pred_labels = []

        label2id = dataset.label2id
        id2label = dataset.id2label
        class_names = [id2label[i] for i in range(len(id2label))]

        # Get predictions
        for i in range(len(dataset)):
            images, true_labels = dataset[i]  # batched data

            preds = model.predict(images)  # shape: (batch_size, num_classes)

            # For multi-class (single-label) classification: use argmax
            pred_ids = np.argmax(preds, axis=1)
            true_ids = np.argmax(true_labels, axis=1)

            all_true_labels.extend(true_ids)
            all_pred_labels.extend(pred_ids)

        # Confusion matrix
        cm = confusion_matrix(all_true_labels, all_pred_labels)
        plt.figure(figsize=(10, 8))
        sns.heatmap(
            cm,
            annot=True,
            fmt="d",
            cmap="Blues",
            xticklabels=class_names,
            yticklabels=class_names,
        )
        plt.title("Confusion Matrix")
        plt.ylabel("True Label")
        plt.xlabel("Predicted Label")
        plt.savefig("confusion_matrix.png", dpi=300, bbox_inches="tight")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Classification report
        report = classification_report(
            all_true_labels, all_pred_labels, target_names=class_names, output_dict=True
        )

        # Log per-class metrics
        for class_name in class_names:
            if class_name in report:
                mlflow.log_metrics(
                    {
                        f"{class_name}_precision": report[class_name]["precision"],
                        f"{class_name}_recall": report[class_name]["recall"],
                        f"{class_name}_f1": report[class_name]["f1-score"],
                    }
                )

    @staticmethod
    def log_evaluation_metrics_MultiLabel(model,dataset):
        """Log comprehensive evaluation metrics."""

        all_true_labels = []
        all_pred_labels = []

        label2id = dataset.label2id
        id2label = dataset.id2label
        class_names = [id2label[i] for i in range(len(id2label))]

        # Get predictions
        for i in range(len(dataset)):
            images, true_labels = dataset[i]  # batched data

            preds = model.predict(images)  # shape: (batch_size, num_classes)

            # For multi-class (single-label) classification: use argmax
            pred_ids = np.argmax(preds, axis=1)
            true_ids = np.argmax(true_labels, axis=1)

            all_true_labels.extend(true_ids)
            all_pred_labels.extend(pred_ids)

        # Confusion matrix
        cm = confusion_matrix(all_true_labels, all_pred_labels)
        plt.figure(figsize=(10, 8))
        sns.heatmap(
            cm,
            annot=True,
            fmt="d",
            cmap="Blues",
            xticklabels=class_names,
            yticklabels=class_names,
        )
        plt.title("Confusion Matrix")
        plt.ylabel("True Label")
        plt.xlabel("Predicted Label")
        plt.savefig("confusion_matrix.png", dpi=300, bbox_inches="tight")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Classification report
        report = classification_report(
            all_true_labels, all_pred_labels, target_names=class_names, output_dict=True
        )

        # Log per-class metrics
        for class_name in class_names:
            if class_name in report:
                mlflow.log_metrics(
                    {
                        f"{class_name}_precision": report[class_name]["precision"],
                        f"{class_name}_recall": report[class_name]["recall"],
                        f"{class_name}_f1": report[class_name]["f1-score"],
                    }
                )






    def model_trainer(self,model,model_trainer_config:ModelTrainerConfig,custom_datagen):
        # You can use 'tensorflow', 'torch', or 'jax' as backend
        # Make sure to set the environment variable before importing Keras
        os.environ["KERAS_BACKEND"] = "tensorflow"


        # Enable autologging for TensorFlow/Keras
        mlflow.tensorflow.autolog()

        experiment_name=model_trainer_config.experiment_name
        run_name=model_trainer_config.run_name
        params = {
        "epochs": model_trainer_config.epochs,
        "batch_size":model_trainer_config.batch_size,
        "learning_rate":model_trainer_config.learning_rate,
        "optimizer":model_trainer_config.optimizer,
        "loss_function":model_trainer_config.loss_function,
        "dropout_rate":model_trainer_config.dropout_rate,
        "hidden_units":model_trainer_config.hidden_units,
        "input_shape":model_trainer_config.input_shape
        }

        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            experiment_id = mlflow.create_experiment(experiment_name)
        else:
            experiment_id = experiment.experiment_id


        with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
        #with mlflow.start_run(nested=True):

            print("Log params")
            # Log training parameters
            mlflow.log_params(params)

            print("Compile the model")
            # Create and compile model
            loss=params["loss_function"]
            if loss == "categorical_crossentropy":
                model.compile(
                    optimizer=keras.optimizers.Adam(learning_rate=params["learning_rate"]),
                    loss=keras.losses.CategoricalCrossentropy(),
                    metrics=["accuracy"],
                )

            elif loss=="binary_crossentropy":
                model.compile(
                    optimizer=keras.optimizers.Adam(learning_rate=params["learning_rate"]),
                    loss=keras.losses.BinaryCrossentropy(),
                    metrics=["accuracy"],
                )

            # get the datasets
            print("Start to preprocess the data ")
            train_dataset = Dataset(self.train_paths, batch_size=params["batch_size"], input_shape=params["input_shape"],datagen=custom_datagen, augment=False)
            val_dataset = Dataset(self.val_paths, batch_size=params["batch_size"], input_shape=params["input_shape"],datagen=custom_datagen, augment=False)
            test_dataset = Dataset(self.test_paths,batch_size=params["batch_size"], input_shape=params["input_shape"],datagen=custom_datagen, augment=False)





            print("save model summery")
            # Log model architecture
            with open("model_summary.txt", "w") as f:
                model.summary(print_fn=lambda x: f.write(x + "\n"))
            mlflow.log_artifact("model_summary.txt")

            with open("base_model_summary.txt", "w") as f:
                model.summary(print_fn=lambda x: f.write(x + "\n"))
            mlflow.log_artifact("base_model_summary.txt")

            print("save model architecture")
            # Log model visualization
            tf.keras.utils.plot_model(model, to_file="model_architecture.png", show_shapes=True)
            mlflow.log_artifact("model_architecture.png")

            print("Define the callback")
            # Define the ModelCheckpoint callback
            directory=save_model_to_artifacts(experiment_id=Path(experiment_name))
            model_path=os.path.join(directory, "model.keras")
            checkpoint = ModelCheckpoint(model_path, save_best_only=True, monitor='val_loss', verbose=1)
            #model.save(model_path)
            ## save
            multilabel_binanzer=train_dataset.mlb
            mlb_path=os.path.join(directory, "mlb.joblib")
            save_joblib(multilabel_binanzer, mlb_path)


            # Custom callback for logging metrics
            class MLflowCallback(keras.callbacks.Callback):
                def on_epoch_end(self, epoch, logs=None):
                    if logs:
                        mlflow.log_metrics(
                            {
                                "train_loss": logs.get("loss"),
                                "train_accuracy": logs.get("accuracy"),
                                "val_loss": logs.get("val_loss"),
                                "val_accuracy": logs.get("val_accuracy"),
                            },
                            step=epoch,
                        )
            # Prepare sample data for signature inference
            sample_input = train_dataset[0][0]
            sample_predictions = model.predict(sample_input)

            # Infer signature from sample data
            signature = infer_signature(sample_input, sample_predictions)

            # Train model with custom callback
            history = model.fit(
                train_dataset,
                batch_size=params["batch_size"],
                epochs=params["epochs"],
                validation_data=val_dataset,
                callbacks=[MLflowCallback(),checkpoint],
                verbose=1,
            )

            # Evaluate model
            ModelTrainer.log_training_plots(history, mlflow.active_run().info.run_id)
            ModelTrainer.log_evaluation_metrics_classification(model,test_dataset)




            #mlflow.keras.log_model(model, name="model",signature=signature)
            #mlflow.keras.log_model(
            #    model,
            #   name="keras_model",
            #   signature=signature,
              #  input_example=sample_input,
              #  registered_model_name="MyKerasHandwrittenDigitRecognizer" # This registers the model
            #)

            if os.path.exists(model_path) and os.path.exists(mlb_path):
              mlflow.log_artifact(model_path)
              print("Model saved in run %s" % mlflow.active_run().info.run_id)
              mlflow.log_artifact(mlb_path)
              print("Model saved in run %s" % mlflow.active_run().info.run_id)

            else:
              print("model not save")

# Step

In [19]:
def config_management_step():
  try:
    config=ConfigManager()
    return config
  except Exception as e:
    raise e


def data_ingestion_step(data_ingestion_config)-> Tuple[List[str], List[str], List[str]]:
  try:
    data_ingestion=DataIngection(data_ingection_config=data_ingestion_config)
    return data_ingestion.get_file_path()
  except Exception as e:
    raise e


def preprocess_step(preprocess_config:PreprocessConfig)-> ImageDataGenerator:
  try:
    custom_datagen = create_datagen_from_config(preprocess_config)
    return custom_datagen
  except Exception as e:
    raise e


def cnn_model_build_step(model_build_config)-> Model:
  try:
    model_build=ModelBuilder(model_build_config)
    return model_build.cnn_model()
  except Exception as e:
    raise e


def vgg16_model_build_step(model_build_config)-> Model:
  try:
    model_build=ModelBuilder(model_build_config)
    return model_build.VGG16_model()
  except Exception as e:
    raise e


def EfficientNetB0_model_build_step(model_build_config)-> Model:
  try:
    model_build=ModelBuilder(model_build_config)
    return model_build.EfficientNetB0_model()
  except Exception as e:
    raise e


def mlflow_connection(mlflow_config: MlflowConfig):
  mlf=MlflowConnection(mlflow_config)
  mlf.mlflow_connection()



def model_trainer_step(train_paths:List, val_paths:List, test_paths:List):
  try:
    model_trainer=ModelTrainer(train_paths, val_paths, test_paths)
    return model_trainer
  except Exception as e:
    raise e

# Pipeline

In [20]:
def model_training_and_eval_pipeline():

    #config step
    print("####### Start config maganer #######")
    config=config_management_step()

    #data ingesstion
    print("####### Start data ingesstion #######")
    data_ingestion_config=config.get_data_ingestion_config()
    train_paths, val_paths, test_paths=data_ingestion_step(data_ingestion_config)

    #preprocess step
    print("####### Start preprocess step #######")
    preprocess_config=config.get_preprocess_config()
    custom_datagen=preprocess_step(preprocess_config)

    #connect dagshub
    print("####### Start mlflow connection #######")
    mlflow_config=config.get_mlflow_config()
    mlflow_connection(mlflow_config)

    #model build step
    print("####### Start model build step #######")
    model_build_config=config.get_model_build_config()


    #model training step
    print("####### Start model training step #######")
    trainer=model_trainer_step(train_paths, val_paths, test_paths)

    #training cnn
    print("####### Start training cnn #######")
    model_cnn=cnn_model_build_step(model_build_config)
    cnn_model_trainer_config=config.get_cnn_model_trainer_config()
    trainer.model_trainer(model_cnn,cnn_model_trainer_config,custom_datagen)


    #training vgg16
    print("####### Start training vgg16 #######")
    model_vgg=vgg16_model_build_step(model_build_config)
    vgg_model_trainer_config=config.get_vgg_model_trainer_config()
    trainer.model_trainer(model_vgg,vgg_model_trainer_config,custom_datagen)


    #training eff
    print("####### Start training eff #######")
    model_eff=EfficientNetB0_model_build_step(model_build_config)
    eff_model_trainer_config=config.get_eff_model_trainer_config()
    trainer.model_trainer(model_eff,eff_model_trainer_config,custom_datagen)


    print("pipeline completed")

In [21]:
model_training_and_eval_pipeline()

####### Start config maganer #######
####### Start data ingesstion #######


libpng warning: iCCP: known incorrect sRGB profile


####### Start preprocess step #######
####### Start mlflow connection #######


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()

...

save model architecture
Define the callback
Joblib file saved to artifacts/document_image_classification/2025-06-29_13-55-18/mlb.joblib
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 58ms/step
Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1923 - loss: 1.7570
Epoch 1: val_loss improved from inf to 1.65050, saving model to artifacts/document_image_classification/2025-06-29_13-55-18/model.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.1912 - loss: 1.7554 - val_accuracy: 0.1797 - val_loss: 1.6505
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 582ms/step - accuracy: 0.2156 - loss: 1.6455
Epoch 2: val_loss improved from 1.65050 to 1.64813, saving model to artifacts/document_image_classification/2025-06-29_13-55-18/model.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 890ms/step - accuracy: 0.2132 - loss: 1.6473 - val_accuracy: 0.1641 - val_loss: 1.6481
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 636ms/step - accuracy: 0.1746 - loss: 1.7251
Epoch 3: val_loss improved from 1.64813 to 1.62411, saving model to ar

2025/06/29 14:07:42 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
libpng warning: iCCP: known incorrect sRGB profile


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model saved in run e7915f71d7e642fa8398a3f5db5ee5f0
Model saved in run e7915f71d7e642fa8398a3f5db5ee5f0
🏃 View run eff_model at: https://dagshub.com/kaushigihanml/document_classification.mlflow/#/experiments/2/runs/e7915f71d7e642fa8398a3f5db5ee5f0
🧪 View experiment at: https://dagshub.com/kaushigihanml/document_classification.mlflow/#/experiments/2
pipeline completed
